In [15]:
output_dir = "./final_model_export"


In [17]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss
import torch

class_weights = torch.tensor([1.5, 1.0]).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [19]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [25]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [26]:
from datasets import Dataset
import pandas as pd


# Load your combined phishing + adversarial dataset
df = pd.read_csv("phishing_combined.csv")
df["Email Text"] = df["Email Text"].astype(str)
df["label"] = df["label"].astype(int)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["Email Text", "label"]])


In [28]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# ✅ Load your combined dataset
combined_df = pd.read_csv("phishing_combined.csv")
combined_df["Email Text"] = combined_df["Email Text"].astype(str)
combined_df["label"] = combined_df["label"].astype(int)

# ✅ Split into train, val, test (70/15/15)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    combined_df["Email Text"], combined_df["label"], test_size=0.30, random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.50, random_state=42
)

# ✅ Rebuild DataFrames
train_df = pd.DataFrame({"Email Text": train_texts, "label": train_labels})
val_df = pd.DataFrame({"Email Text": val_texts, "label": val_labels})
test_df = pd.DataFrame({"Email Text": test_texts, "label": test_labels})

# ✅ Convert to HF Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print("✅ Datasets ready:", train_dataset.shape, val_dataset.shape, test_dataset.shape)


✅ Datasets ready: (4, 3) (1, 3) (1, 3)


In [31]:
# 🔁 Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["Email Text"], padding="max_length", truncation=True)

# ✅ Apply tokenizer to each split
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# ✅ Set dataset format to PyTorch
train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [33]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, DataCollatorWithPadding, Trainer
from torch.nn import CrossEntropyLoss
import torch

# ✅ Model + tokenizer
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Weighted loss
class_weights = torch.tensor([1.5, 1.0]).to(device)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


# ✅ Training args
training_args = TrainingArguments(
    output_dir="./results_adv_retrain_test",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    load_best_model_at_end=True,
    report_to="none"
)

# ✅ Train (tiny dataset for now)
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.658900,0.637072
2,0.550200,0.529761
3,0.395200,0.501151


TrainOutput(global_step=6, training_loss=0.5749296446641287, metrics={'train_runtime': 64.0261, 'train_samples_per_second': 0.187, 'train_steps_per_second': 0.094, 'total_flos': 1589608783872.0, 'train_loss': 0.5749296446641287, 'epoch': 3.0})

In [34]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")



('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [35]:
!zip -r model.zip model
from google.colab import files
files.download("model.zip")


  adding: model/ (stored 0%)
  adding: model/tokenizer.json (deflated 71%)
  adding: model/vocab.txt (deflated 53%)
  adding: model/tokenizer_config.json (deflated 75%)
  adding: model/config.json (deflated 45%)
  adding: model/model.safetensors (deflated 8%)
  adding: model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
# ✅ Save the full model and tokenizer to 'model' directory
model.save_pretrained("model")
tokenizer.save_pretrained("model")


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [37]:
# ✅ Zip the folder
!zip -r model.zip model

# ✅ Download it to your computer
from google.colab import files
files.download("model.zip")


updating: model/ (stored 0%)
updating: model/tokenizer.json (deflated 71%)
updating: model/vocab.txt (deflated 53%)
updating: model/tokenizer_config.json (deflated 75%)
updating: model/config.json (deflated 45%)
updating: model/model.safetensors (deflated 8%)
updating: model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>